## Data Cleaning & Advance Analysis

In [1]:
import pandas as pd
import numpy as np

In [2]:
# Load dataset
df = pd.read_csv(r"E:\Major Projects\Zomato Geospatial Case Study\data\raw\zomato.csv")

In [3]:
print(df.shape)
df.head()

(51717, 17)


,url,address,name,online_order,book_table,rate,votes,phone,location,rest_type,dish_liked,cuisines,approx_cost(for two people),reviews_list,menu_item,listed_in(type),listed_in(city)
0,https://www.zomato.com/bangalore/jalsa-banasha...,"942, 21st Main Road, 2nd Stage, Banashankari, ...",Jalsa,Yes,Yes,4.1/5,775,080 42297555\r\n+91 9743772233,Banashankari,Casual Dining,"Pasta, Lunch Buffet, Masala Papad, Paneer Laja...","North Indian, Mughlai, Chinese",800,"[('Rated 4.0', 'RATED\n A beautiful place to ...",[],Buffet,Banashankari
1,https://www.zomato.com/bangalore/spice-elephan...,"2nd Floor, 80 Feet Road, Near Big Bazaar, 6th ...",Spice Elephant,Yes,No,4.1/5,787,080 41714161,Banashankari,Casual Dining,"Momos, Lunch Buffet, Chocolate Nirvana, Thai G...","Chinese, North Indian, Thai",800,"[('Rated 4.0', 'RATED\n Had been here for din...",[],Buffet,Banashankari
2,https://www.zomato.com/SanchurroBangalore?cont...,"1112, Next to KIMS Medical College, 17th Cross...",San Churro Cafe,Yes,No,3.8/5,918,+91 9663487993,Banashankari,"Cafe, Casual Dining","Churros, Cannelloni, Minestrone Soup, Hot Choc...","Cafe, Mexican, Italian",800,"[('Rated 3.0', ""RATED\n Ambience is not that ...",[],Buffet,Banashankari
3,https://www.zomato.com/bangalore/addhuri-udupi...,"1st Floor, Annakuteera, 3rd Stage, Banashankar...",Addhuri Udupi Bhojana,No,No,3.7/5,88,+91 9620009302,Banashankari,Quick Bites,Masala Dosa,"South Indian, North Indian",300,"[('Rated 4.0', ""RATED\n Great food and proper...",[],Buffet,Banashankari
4,https://www.zomato.com/bangalore/grand-village...,"10, 3rd Floor, Lakshmi Associates, Gandhi Baza...",Grand Village,No,No,3.8/5,166,+91 8026612447\r\n+91 9901210005,Basavanagudi,Casual Dining,"Panipuri, Gol Gappe","North Indian, Rajasthani",600,"[('Rated 4.0', 'RATED\n Very good restaurant ...",[],Buffet,Banashankari


In [4]:
df = df.rename(columns={
    'name':'restaurant_name',
    'rate':'rating',
    'approx_cost(for two people)':'cost_for_two',
    'listed_in(type)':'service_type'
})

In [5]:
#Cleaning the rating col
def clean_rating(value):
    if isinstance(value, str):
        v = value.strip()
        if v in ['NEW', '-', 'nan', 'NaN', '']:
            return np.nan
        if '/5' in v:
            try:
                return float(v.split('/')[0])
            except:
                return np.nan
        try:
            return float(v)
        except:
            return np.nan
    return np.nan

df['rating_clean'] = df['rating'].apply(clean_rating)


In [6]:
# cleaning the cost for two col
def clean_cost(x):
    try:
        return float(str(x).replace(',', '').strip())
    except:
        return np.nan

df['cost_for_two_clean'] = df['cost_for_two'].apply(clean_cost)


In [7]:
# convert yes/no fields to binary flags
df['online_order_flag'] = df['online_order'].apply(lambda x: 1 if str(x).strip().lower()=='yes' else 0)
df['book_table_flag'] = df['book_table'].apply(lambda x: 1 if str(x).strip().lower()=='yes' else 0)


In [8]:
# dropping useless duplicate files
# Drop rows where both rating and cost are missing
df = df.dropna(subset=['rating_clean', 'cost_for_two_clean'], how='all')

# Remove duplicates (if any)
df = df.drop_duplicates(subset=['restaurant_name','location','cost_for_two_clean'])


In [9]:
# assigning unique IDs
df = df.reset_index(drop=True)
df['restaurant_id'] = df.index + 1


In [10]:
# saving cleaned files
zomato_clean = df[['restaurant_id','restaurant_name','location',
                   'rating_clean','votes','cost_for_two_clean',
                   'service_type','online_order','book_table','cuisines']]

zomato_clean.to_csv(r"E:\Major Projects\Zomato Geospatial Case Study\data\raw\processed\cleaned\zomato_clean.csv", index=False)

In [11]:
df = pd.read_csv(r"E:\Major Projects\Zomato Geospatial Case Study\data\raw\processed\cleaned\zomato_clean.csv")

In [12]:
df['cuisines'] = df['cuisines'].fillna("").str.split(",")
df_cuisines = df.explode('cuisines')

In [13]:
df_cuisines['cuisines'] = df_cuisines['cuisines'].str.strip()

In [14]:
df_cuisines.to_csv(r"E:\Major Projects\Zomato Geospatial Case Study\data\raw\processed\cleaned\RestaurantCuisines.csv", index = False)
print("Saved")

Saved


## Geocoding (Geospatial Analysis)

In [15]:
pip install geopandas

Note: you may need to restart the kernel to use updated packages.


In [16]:
import geopandas as gpd
import pyodbc

In [17]:
!pip install geopy --user

In [18]:
import sys
print(sys.executable)

E:\anaconda3\python.exe


In [19]:
!{sys.executable} -m pip install geopy

In [20]:
import geopy
print(geopy.__version__)

2.4.1


In [21]:
from geopy.geocoders import Nominatim
from geopy.extra.rate_limiter import RateLimiter

In [22]:
# Use unique locations to avoid duplicate API calls
unique_locations = df['location'].dropna().unique()

In [23]:
geolocator = Nominatim(user_agent="zomato_geocoder")
geocode = RateLimiter(geolocator.geocode, min_delay_seconds=1)

In [ ]:
# Collect results
location_data = []
for loc in unique_locations:
    try:
        g = geocode(loc + ", Bangalore, India")  # add city for better accuracy
        if g:
            location_data.append({
                "location": loc,
                "latitude": g.latitude,
                "longitude": g.longitude
            })
        else:
            location_data.append({
                "location": loc,
                "latitude": None,
                "longitude": None
            })
    except Exception as e:
        print(f"Error for {loc}: {e}")
        location_data.append({
            "location": loc,
            "latitude": None,
            "longitude": None
        })

In [25]:
# Save locations to CSV
locations_df = pd.DataFrame(location_data)
locations_df.to_csv(r"E:\Major Projects\Zomato Geospatial Case Study\data\raw\processed\cleaned\locations.csv", index=False)

print("✅ Geocoding completed & saved to locations.csv")

✅ Geocoding completed & saved to locations.csv


In [26]:
conn = pyodbc.connect(
    "Driver={ODBC Driver 17 for SQL Server};"
    "Server=LAPTOP-KT5S5Q1A\\SQLEXPRESS;"
    "Database=Zomato_db;"
    "Trusted_Connection=yes;"
    "MARS_Connection=Yes;"
)

In [ ]:
Restaurants = pd.read_sql_query("""
    SELECT RestaurantID, Name, Location, AverageCostForTwo, PriceRange,
           AggregateRating, Votes, Latitude, Longitude
    FROM Restaurants
""", conn)

In [28]:
import geopandas as gpd
from shapely.geometry import Point

In [29]:
# Create geometry column from lat/lon
geometry = [Point(xy) for xy in zip(Restaurants['Longitude'], Restaurants['Latitude'])]

# Build GeoDataFrame
gdf = gpd.GeoDataFrame(Restaurants, geometry=geometry, crs="EPSG:4326")

print(gdf.head())

   RestaurantID                   Name      Location  AverageCostForTwo  \
0             1                  Jalsa  Banashankari              800.0   
1             2         Spice Elephant  Banashankari              800.0   
2             3        San Churro Cafe  Banashankari              800.0   
3             4  Addhuri Udupi Bhojana  Banashankari              300.0   
4             5          Grand Village  Basavanagudi              600.0   

  PriceRange  AggregateRating  Votes   Latitude  Longitude  \
0       None              4.1    775  12.927819  77.556618   
1       None              4.1    787  12.927819  77.556618   
2       None              3.8    918  12.927819  77.556618   
3       None              3.7     88  12.927819  77.556618   
4       None              3.8    166  12.941726  77.575500   

                    geometry  
0  POINT (77.55662 12.92782)  
1  POINT (77.55662 12.92782)  
2  POINT (77.55662 12.92782)  
3  POINT (77.55662 12.92782)  
4   POINT (77.5755 12

In [30]:
import plotly.express as px

In [ ]:
# Plot restaurants as scatter mapbox
fig = px.scatter_mapbox(
    gdf,
    lat="Latitude",
    lon="Longitude",
    color="AggregateRating",   # color by rating
    size="Votes",              # bubble size by votes
    hover_name="Name",
    hover_data=["Location", "AverageCostForTwo"],
    zoom=11,
    height=600
)

fig.update_layout(mapbox_style="open-street-map")
fig.show()

In [ ]:
fig = px.density_mapbox(
    gdf,
    lat="Latitude",
    lon="Longitude",
    z=None,  # just counts density
    radius=15,
    center=dict(lat=gdf["Latitude"].mean(), lon=gdf["Longitude"].mean()),
    zoom=11,
    mapbox_style="open-street-map",
    title="Restaurant Density Heatmap"
)
fig.show()


In [ ]:
top_cuisines = df_cuisines['cuisines'].value_counts().nlargest(10)

fig = px.bar(
    top_cuisines,
    x=top_cuisines.index,
    y=top_cuisines.values,
    title="Top 10 Cuisines in Bangalore",
    labels={"x":"Cuisine", "y":"Count"}
)
fig.show()


In [ ]:
fig = px.scatter(
    gdf,
    x="AverageCostForTwo",
    y="AggregateRating",
    size="Votes",
    color="Location",
    hover_name="Name",
    title="Price vs Rating of Restaurants"
)
fig.show()


In [ ]:
import plotly.express as px

# Count restaurants per location
location_stats = Restaurants.groupby("Location").agg({
    "RestaurantID": "count",
    "Votes": "sum",
    "AggregateRating": "mean"
}).reset_index()

location_stats.rename(columns={
    "RestaurantID": "Num_Restaurants",
    "Votes": "Total_Votes",
    "AggregateRating": "Avg_Rating"
}, inplace=True)

# Define "underserved" as: low restaurants but high votes (demand > supply)
location_stats["Demand_Supply_Gap"] = location_stats["Total_Votes"] / (location_stats["Num_Restaurants"] + 1)

# Sort top 15 underserved locations
underserved = location_stats.sort_values("Demand_Supply_Gap", ascending=False).head(15)

# Bar chart
fig = px.bar(
    underserved,
    x="Location",
    y="Demand_Supply_Gap",
    color="Avg_Rating",
    title="Top 15 Underserved Locations (High Demand, Low Supply)",
    labels={"Demand_Supply_Gap": "Votes per Restaurant"},
    text="Num_Restaurants"
)
fig.update_traces(textposition="outside")
fig.show()
